In [1]:
import langchain

In [2]:
import openai

In [3]:
import json

In [4]:
mykey="sk-proj-JlmfKSTRyMWIJi77m6uUT3BlbkFJxKzJY33qI00b9RCfktG7"

In [5]:
from langchain import OpenAI
client = OpenAI(api_key = mykey)

C:\Users\HarshaVardhan Reddy\anaconda3\envs\testopenai\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chains import SequentialChain

Patient will input the following details :
 • Symptoms experienced by the patient
 • Age of the patient
 • City of residence
 • Severity level of the symptoms
 • The patient will also provide the duration of their symptoms, indicating the number of 
days they have been experiencing them

In [7]:
symptoms_of = PromptTemplate(
    input_variables=["symptoms"],
    template="Can you find the disease according to these symptoms: {symptoms}?"
)

chain1 = LLMChain(llm=client, prompt=symptoms_of, output_key="symptoms_out")

age_of = PromptTemplate(
    input_variables=["symptoms_out", "age"],
    template="Can you guess the health condition of the patient using these symptoms: {symptoms_out} and age: {age}?"
)

chain2 = LLMChain(llm=client, prompt=age_of, output_key="age_out")

city_of = PromptTemplate(
    input_variables=["age_out", "city"],
    template="Can you give me recommendations of hospitals the patient can visit for treatment using this information: age: {age_out} and symptoms: {symptoms_out}, and the current location of the patient: {city}?"
)

chain3 = LLMChain(llm=client, prompt=city_of, output_key="city_out")

severity_of = PromptTemplate(
    input_variables=["city_out", "severity"],
    template="Severity level of the symptoms: {city_out} and severity: {severity}"
)

chain4 = LLMChain(llm=client, prompt=severity_of, output_key="severity_out")

duration_of = PromptTemplate(
    input_variables=["severity_out", "duration"],
    template="Since {duration}, the person is suffering from this: {severity_out}. "
)

chain5 = LLMChain(llm=client, prompt=duration_of, output_key="duration_out")

chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4, chain5],
    input_variables=["symptoms", "age", "city", "severity", "duration"],
    output_variables=["duration_out"]
)


C:\Users\HarshaVardhan Reddy\anaconda3\envs\testopenai\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
res = chain({"symptoms": "high body temperature and stomach pain and loose motions", "age": "24", "city": "hydrabad", "severity": "medium stage" , "duration":"two weeks"})


In [11]:
res


{'symptoms': 'high body temperature and stomach pain and loose motions',
 'age': '24',
 'city': 'hydrabad',
 'severity': 'medium stage',
 'duration': 'two weeks',
 'duration_out': '\n\nIt is unclear what the person may be suffering from as no specific symptoms or condition is mentioned. It is recommended to consult a healthcare professional for a proper diagnosis and treatment plan.'}